Political Influence on Stock Market Performance Analysis

In [ ]:
!pip install yfinance

In [ ]:
!pip install --upgrade yfinance

In [ ]:
import pandas as pd
import yfinance as yf

sp500 = yf.download('^GSPC', start = '1920-01-17', end='2025-03-11')

sp500.to_csv('sp500.csv')

sp500df = pd.read_csv('sp500.csv', index_col='Price', parse_dates=True)
sp500df.head()

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime

start = datetime(2024, 1, 10)
end = datetime(2025, 6, 1)

# Fetch data using yfinance
data = yf.download('^GSPC', start=start, end=end)

# Add dummy variable: 1 if Close > Open = 1, else 0
data[('dummy_var')] = (data[('Close', '^GSPC')] > data[('Open', '^GSPC')]).astype(int)
data

In [ ]:
data['Low']

In [ ]:
# Drop NA and select correct columns
clean_data = data[[('Close', '^GSPC'), ('dummy_var', '')]].dropna()
#clean_data
corr = clean_data[('Close', '^GSPC')].corr(clean_data[('dummy_var', '')])
corr


In [ ]:
data.isnull().sum()

In [ ]:
data[['Volume']].plot()

In [ ]:
start = datetime(2025, 2, 17)
end = datetime(2025, 3, 11)

# Fetch data using yfinance
data = yf.download('AAPL', start=start, end=end)
data

In [ ]:
start = datetime(2025, 2, 17)
end = datetime(2025, 3, 28)

# Fetch data using yfinance
data = yf.download('^IXIC', start=start, end=end)
data.head()

In [ ]:
self._set_session(session or requests.Session(impersonate="chrome"))

In [ ]:
  from curl_cffi import requests
  session = requests.Session(impersonate="chrome")
  ticker = yf.Ticker('^IXIC', session=session)

In [ ]:
Първо емпиричната част
с 0 и 1 - регресионен модел(като фактор), може да се добави и обема като фактор

1.Информация за индексите /Сваляне на данните
Уточнаване на събития
2. Информация за анализите или политически събития; почистване. спад, скок
    Подготовка на данните, графики, описания на динамиката; процент на прираст
    Трениране на модел
3. Метода с който правя нещо, 
4. поне 9 месеца
